In [1]:
# # 패키지 설치 (필요 시 실행)

# %pip install -U langchain langchain-text-splitters langchain-community langchain-openai langchain-pinecone pinecone langgraph python-dotenv pypdf

# Assignment 04 — 02_nist_rag_agent.ipynb

이 노트북은 01의 RAG 체인을 LangChain Tool로 래핑하고, 최신 Agent + Memory를 구성한 뒤 LangGraph로 호출 흐름을 감싸 LangSmith에 트레이스를 남깁니다.

In [2]:
# 10) 환경/키 재설정 및 공통 유틸 로드
import os, json
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_community.vectorstores import Pinecone # Deprecated
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# LangSmith tracing (01번 노트북과 동일 프로젝트 사용)
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING", "true")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY", "")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT", "nist-rag-project")

# reuse index / namespace / embeddings (01_nist_rag와 동일하게)
INDEX_NAME = "nist-rag-index"
NAMESPACE = "nist"

# 01번 노트북과 동일하게 1024 차원 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1024)

# PineconeVectorStore 초기화 (from_existing_index 사용 권장)
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings,
    namespace=NAMESPACE
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("VectorStore ready for Agent (index=", INDEX_NAME, ", namespace=", NAMESPACE, ")")

/home/dhc99/anaconda3/envs/torch_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStore ready for Agent (index= nist-rag-index , namespace= nist )


In [3]:
# 11) RAG 체인 Tool 래핑 (@tool 또는 Tool)
from langchain.tools import tool

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 NIST 기반 AI/보안 컨설턴트다. 이 Tool은 NIST 문서를 검색해 근거를 제공한다. 어떤 문서/페이지를 참고했는지 한 문장으로 먼저 언급하고, 모르면 모른다고 답해라."),
    ("human", "질문: {question}\n\n다음은 관련 문서 청크들이다:\n{context}\n\n위 청크들만을 근거로, 한국어로 자세히 답변해라."),
])


def format_docs(docs):
    parts = []
    for d in docs:
        m = d.metadata
        parts.append(f"[source={m.get('source')}, doc_title={m.get('doc_title')}, page={m.get('page')}]\n{d.page_content}")
    return "\n\n".join(parts)

rag_chain = (
    {"context": retriever | format_docs, "question": lambda x: x}
    | prompt
    | llm
    | StrOutputParser()
)

@tool("nist_rag_tool")
def nist_rag_tool(question: str) -> str:
    """질문을 받아 NIST 문서 근거(문서명/페이지)를 포함한 답변을 생성한다."""
    # invoke() 사용
    return rag_chain.with_config({
        "tags": ["nist", "rag", "tool"],
        "run_name": "nist_rag_tool"
    }).invoke(question)

print("Tool ready: nist_rag_tool")

Tool ready: nist_rag_tool


In [4]:
# 12) Memory 구성 (ConversationBufferMemory)
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)
print("Memory ready")

Memory ready


/tmp/ipykernel_2872242/3025415612.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [5]:
# 13) 최신 Agent 생성 및 System 프롬프트
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Agent용 시스템 프롬프트: 반드시 RAG Tool 우선 사용
sys_prompt = (
    "너는 NIST 기반 AI/보안 컨설턴트다. 가능하면 항상 'nist_rag_tool'을 먼저 사용해 답변하라. "
    "답변 시작 부분에서 어떤 문서/페이지를 참고했는지 한 문장으로 요약하고, 모르면 모른다고 답해라."
)

# Tool Calling Agent Prompt (OpenAI Tools 호환)
prompt = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    # Memory 사용 시 history placeholder 필요할 수 있으나, 
    # AgentExecutor가 memory를 관리하므로 여기서는 input/scratchpad만 필수
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [nist_rag_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

print("Agent ready with nist_rag_tool and memory")

Agent ready with nist_rag_tool and memory


In [6]:
# 14) 대화 시나리오 1: AI Risk Management 컨설팅 (3–5턴)
scenario1 = [
    "AI RMF의 목표를 한 문장으로 설명해줘",
    "데이터 거버넌스는 AI RMF에서 어떻게 다뤄져?",
    "리스크 모니터링과 보고의 핵심 활동은?",
]

for msg in scenario1:
    resp = agent_executor.with_config({
        "tags": ["nist", "agent"],
        "run_name": "scenario_ai_rmf"
    }).invoke({"input": msg})
    print(resp["output"])



> Entering new scenario_ai_rmf chain...

Invoking: `nist_rag_tool` with `{'question': 'AI RMF의 목표는 무엇인가요?'}`



Invoking: `nist_rag_tool` with `{'question': 'AI RMF의 목표는 무엇인가요?'}`


AI RMF의 목표는 AI 시스템을 설계, 개발, 배포 또는 사용하는 조직들이 AI의 다양한 위험을 관리하고 신뢰할 수 있으며 책임 있는 AI 시스템의 개발과 사용을 촉진하는 데 도움을 주는 자원을 제공하는 것입니다. 이 프레임워크는 자발적이며, 권리를 존중하고, 특정 산업에 국한되지 않으며, 사용 사례에 구애받지 않도록 설계되어 있어 모든 규모와 모든 분야의 조직들이 이 프레임워크의 접근 방식을 구현할 수 있는 유연성을 제공합니다. (출처: AI RMF, 페이지 6.0)AI RMF의 목표는 AI 시스템을 설계, 개발, 배포 또는 사용하는 조직들이 AI의 다양한 위험을 관리하고 신뢰할 수 있으며 책임 있는 AI 시스템의 개발과 사용을 촉진하는 데 도움을 주는 자원을 제공하는 것입니다. 이 프레임워크는 자발적이며, 권리를 존중하고, 특정 산업에 국한되지 않으며, 사용 사례에 구애받지 않도록 설계되어 있어 모든 규모와 모든 분야의 조직들이 이 프레임워크의 접근 방식을 구현할 수 있는 유연성을 제공합니다. (출처: AI RMF, 페이지 6.0)AI RMF의 목표는 AI 시스템을 설계, 개발, 배포 또는 사용하는 조직들이 AI의 다양한 위험을 관리하고 신뢰할 수 있으며 책임 있는 AI 시스템의 개발과 사용을 촉진하는 데 도움을 주는 자원을 제공하는 것입니다. (출처: AI RMF, 페이지 6.0)

> Finished chain.
AI RMF의 목표는 AI 시스템을 설계, 개발, 배포 또는 사용하는 조직들이 AI의 다양한 위험을 관리하고 신뢰할 수 있으며 책임 있는 AI 시스템의 개발과 사용을 촉진하는 데 도움을 주는 자원을 제공하는 것입니다.

In [7]:
# 15) 대화 시나리오 2: Zero Trust 전환 컨설팅 (3–5턴)
scenario2 = [
    "Zero Trust의 핵심 원칙을 요약해줘",
    "자산 접근 제어는 어떤 방식으로 구현해야 해?",
    "지속적인 검증(continuous verification)은 무엇을 의미해?",
]

for msg in scenario2:
    resp = agent_executor.with_config({
        "tags": ["nist", "agent"],
        "run_name": "scenario_zero_trust"
    }).invoke({"input": msg})
    print(resp["output"])



> Entering new scenario_zero_trust chain...

Invoking: `nist_rag_tool` with `{'question': 'Zero Trust의 핵심 원칙은 무엇인가요?'}`



Invoking: `nist_rag_tool` with `{'question': 'Zero Trust의 핵심 원칙은 무엇인가요?'}`


참고한 문서: NIST SP 800-207, Zero Trust, page 9.0 및 12.0.

Zero Trust의 핵심 원칙은 다음과 같습니다. Zero Trust는 자원 보호에 중점을 두고 있으며, 신뢰는 암묵적으로 부여되지 않고 지속적으로 평가되어야 한다는 전제를 가지고 있습니다. 이는 모든 엔터프라이즈 자산(장치, 인프라 구성 요소, 애플리케이션, 가상 및 클라우드 구성 요소 등)과 주체(최종 사용자, 애플리케이션 및 정보 요청을 하는 비인간 엔티티)를 포함하는 포괄적인 접근 방식을 요구합니다. 

Zero Trust 보안 모델은 환경 내에 공격자가 존재한다고 가정하며, 기업 소유 환경이 비기업 소유 환경보다 더 신뢰할 수 있다고 보지 않습니다. 따라서 기업은 암묵적인 신뢰를 가정하지 않고, 자원에 대한 접근을 필요로 하는 주체에게 최소한의 권한(예: 읽기, 쓰기, 삭제)만을 부여해야 합니다. 이러한 접근 방식은 내부 네트워크에 접근한 인증된 주체가 광범위한 자원에 대한 권한을 부여받는 전통적인 방식과는 대조적입니다. 

결론적으로, Zero Trust는 자원과 데이터의 보안을 위한 종합적인 접근 방식으로, 신뢰를 지속적으로 평가하고 최소한의 권한만을 부여하는 것을 핵심 원칙으로 삼고 있습니다.참고한 문서: NIST SP 800-207, Zero Trust, page 9.0 및 12.0.

Zero Trust의 핵심 원칙은 다음과 같습니다. Zero Trust는 자원 보호에 중점을 두고 있으며, 신뢰는 암묵적으로 부여되지 않고 지속적으로 평가되어야 한다는 전제를 가지고 있습니다. 이는 모든 엔터

In [8]:
# 16) LangGraph로 Agent 호출 그래프 정의/컴파일
from langgraph.graph import StateGraph, MessagesState, END
from langchain_core.messages import HumanMessage, AIMessage

def agent_node(state: MessagesState):
    # 마지막 user 메시지를 Agent에 전달
    messages = state["messages"]
    last_msg = messages[-1]
    user_content = last_msg.content if hasattr(last_msg, "content") else last_msg["content"]
    
    # AgentExecutor 실행
    out = agent_executor.with_config({
        "tags": ["nist", "agent", "langgraph"],
        "run_name": "agent_node"
    }).invoke({"input": user_content})
    
    # messages 리스트에 assistant 응답 추가 (LangGraph reducer가 처리)
    return {"messages": [AIMessage(content=out["output"])]}

builder = StateGraph(MessagesState)
builder.add_node("agent_node", agent_node)
# 단순 실행: Start -> Agent -> End
builder.add_edge("agent_node", END)
builder.set_entry_point("agent_node")

graph = builder.compile()
print("Graph compiled.")

Graph compiled.


In [9]:
# 17) LangGraph 실행 및 LangSmith 트레이스 태깅
init_state = {"messages": [{"role": "user", "content": "AI RMF의 핵심 목표는?"}]}

# single invoke for demo
out_state = graph.invoke(init_state, config={
    "tags": ["nist", "agent", "langgraph"],
    "run_name": "nist_agent_graph"
})
print(out_state)



> Entering new agent_node chain...

Invoking: `nist_rag_tool` with `{'question': 'AI RMF의 핵심 목표는 무엇인가요?'}`



Invoking: `nist_rag_tool` with `{'question': 'AI RMF의 핵심 목표는 무엇인가요?'}`


AI RMF의 핵심 목표는 AI 위험을 효과적으로 관리하고, 이를 통해 조직의 목표와 법적/규제 요구사항, 그리고 모범 사례에 잘 부합하도록 하는 것입니다. AI RMF는 현재 상태(Current Profile)와 목표 상태(Target Profile)를 비교하여 AI 위험 관리 활동의 격차를 식별하고, 이를 해결하기 위한 행동 계획을 개발하는 과정을 포함합니다. 이러한 목표를 달성하기 위해서는 특정 산업이나 조직의 맥락에서 AI 위험 관리의 우선순위를 반영해야 하며, 이를 통해 원하는 결과를 얻기 위한 목표를 설정하고 달성하는 것이 중요합니다. (출처: AI RMF, 페이지 37.0)AI RMF의 핵심 목표는 AI 위험을 효과적으로 관리하고, 이를 통해 조직의 목표와 법적/규제 요구사항, 그리고 모범 사례에 잘 부합하도록 하는 것입니다. AI RMF는 현재 상태(Current Profile)와 목표 상태(Target Profile)를 비교하여 AI 위험 관리 활동의 격차를 식별하고, 이를 해결하기 위한 행동 계획을 개발하는 과정을 포함합니다. 이러한 목표를 달성하기 위해서는 특정 산업이나 조직의 맥락에서 AI 위험 관리의 우선순위를 반영해야 하며, 이를 통해 원하는 결과를 얻기 위한 목표를 설정하고 달성하는 것이 중요합니다. (출처: AI RMF, 페이지 37.0)AI RMF의 핵심 목표는 AI 위험을 효과적으로 관리하고, 조직의 목표 및 법적/규제 요구사항에 부합하도록 하는 것입니다. (출처: AI RMF, 페이지 37.0)

> Finished chain.
{'messages': [HumanMessage(content='AI RM

In [10]:
# 18) Trace 요약/검증용 결과 수집 코드
# (Placeholder) In a real run, query LangSmith API or parse local logs.
summary = {
    "tool_calls": "(collect from LangSmith traces)",
    "turns_with_tool": "(collect by run_name 'nist_rag_tool')",
    "graph_nodes_order": ["agent_node"],
    "latency_token_usage": "(collect from trace metadata)",
}

out_dir = Path("assignment04/results")
out_dir.mkdir(parents=True, exist_ok=True)
with open(out_dir / "agent_graph_summary.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)
print("Saved agent/graph summary to", out_dir / "agent_graph_summary.json")

Saved agent/graph summary to assignment04/results/agent_graph_summary.json
